In [2]:
#!/usr/bin/env python3

from ManipulaPy.urdf_processor import URDFToSerialManipulator
from math import pi
import numpy as np


In [3]:
# Path to your URDF file
urdf_file_path = "ur5/ur5/ur5.urdf"

# Initialize the URDFToSerialManipulator with the URDF file
urdf_processor = URDFToSerialManipulator(urdf_file_path)

# Extract the SerialManipulator object
ur5 = urdf_processor.serial_manipulator

# Screw Axis Theory in Robotics

Screw axis theory is a fundamental concept in robotics and mechanical engineering, providing a powerful framework for describing and analyzing the motion of rigid bodies in three-dimensional space. This theory is particularly useful in the context of robotic manipulators, where understanding and controlling the motion of joints and links is crucial.

## Understanding Screw Axis

A screw motion in three-dimensional space can be thought of as a combination of a rotation about an axis and a translation along the same axis. This concept is encapsulated in the screw axis, which is defined by the following parameters:

- **Axis of Rotation**: A line in space about which the rotation occurs.
- **Angle of Rotation**: The magnitude of rotation about the axis.
- **Direction of Translation**: The direction along the axis in which the translation occurs.
- **Magnitude of Translation**: The distance moved along the axis.

## Representation of Screw Axis

In robotics, the screw axis is often represented as a 6-dimensional vector, known as a twist. The twist vector combines the angular velocity (representing rotation) and linear velocity (representing translation) of a rigid body:

$$
\text{T} = \begin{bmatrix} \omega \\ v \end{bmatrix}
$$
where:
- $\omega$ is a 3-dimensional vector representing the angular velocity.
- $v$ is a 3-dimensional vector representing the linear velocity.

# Twist Expression in Terms of Screw Axis

The twist vector associated with motion about a screw axis can be expressed in terms of the screw axis parameters:

- **Unit Vector ( $\hat{s}$ )**: A unit vector along the screw axis, representing the direction of the axis.
- **Point on the Axis \( $q$ \)**: A point through which the screw axis passes.
- **Screw Pitch \( $h$ \)**: The ratio of translation to rotation along the screw axis.

Given these parameters and an angular velocity  $\theta_{\dot{}}$ about the screw axis, the twist $V$ can be expressed as:

$$
V = \begin{bmatrix} \omega \\ v \end{bmatrix} = \begin{bmatrix} \hat{s} \theta_{\dot{}} \\ (q \times \hat{s}) \theta_{\dot{}} + h \hat{s} \theta_{\dot{}} \end{bmatrix}
$$

Here:
-  $\omega$ = $\hat{s}$ $\theta_{\dot{}}$  is the angular velocity component of the twist, representing rotation about the screw axis.
- $v$ = -$\hat{s}$ $\theta_{\dot{}}$   $\times$ $q$ + $h$ $\hat{s}$ $\theta_{\dot{}}$  is the linear velocity component of the twist, representing translation along the screw axis.


## Mathematical Tools

### Matrix Exponential and Logarithm
- **Matrix Exponential (`MatrixExp6`)**: Converts a twist (screw axis representation) into a transformation matrix, which is used to describe the pose of a rigid body.
- **Matrix Logarithm (`MatrixLog6`)**: The inverse operation, used to convert a transformation matrix back into a twist.

These mathematical tools are key to applying screw theory in practical robotics applications, enabling the conversion between different representations of motion and facilitating the computation of robot dynamics and kinematics.


# Forward Kinematics Using Screw Axis Theory

Forward kinematics in robotics is the process of determining the position and orientation of the end-effector given the joint parameters (like angles for revolute joints or displacements for prismatic joints). Screw axis theory provides a powerful framework for analyzing and computing forward kinematics, especially for robotic arms.

## Screw Axis Theory in Robotics

Screw axis theory describes the motion of a rigid body as a combination of rotation and translation along a screw axis. This motion can be represented by a twist vector:

$$
\xi = \begin{bmatrix} \omega \\ v \end{bmatrix}
$$

where $( \omega )$ is the angular velocity vector and $( v )$ is the linear velocity vector.

## Application in Forward Kinematics

In the context of forward kinematics, each joint of a robotic manipulator can be associated with a screw axis. The motion of the joint results in a rigid body transformation that can be described using the exponential map of the twist associated with the joint.

### Exponential Map of a Twist

The exponential map of a twist $( x_i )$ for a joint with an angle $( \theta )$ is given by:

$$
T = \exp(\hat{\xi} \theta)
$$

where $( \hat{\xi} )$ is the twist in matrix form and $( \exp )$ denotes the matrix exponential.

### Computing the End-Effector Pose

The pose of the end-effector is obtained by chaining the transformations of each joint:

$$
T_{\text{end-effector}} = T_1 T_2 \cdots T_n
$$

where $( T_i )$ is the transformation matrix for the \( i \)-th joint, computed using the exponential map of its associated twist.



In [4]:
# Example joint angles (thetalist) for the manipulator
thetalist = np.array([pi, pi/6, pi/4, -pi/3, -pi/2, (-2*pi/3)])
dthetalist = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
ddthetalist = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])


In [5]:
T=ur5.forward_kinematics(thetalist,frame='space')
print(T)


[[   -0.22414     0.96593     0.12941     0.04353]
 [        0.5 -2.2931e-16     0.86603     -1.1624]
 [    0.83652     0.25882    -0.48296     -1.3458]
 [          0           0           0           1]]


In [6]:
ur5_d=urdf_processor.dynamics
tau=ur5_d.inverse_dynamics(thetalist, dthetalist, ddthetalist, [0,0,-9.81], [0]*6)
urdf_processor.visualize_robot()

2025-04-14 16:00:18 - OpenGL.acceleratesupport - INFO - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [7]:
from ManipulaPy.path_planning import TrajectoryPlanning as tp


Joint_limits = [
    (-np.pi, np.pi),  # Joint 1
    (-np.pi/2, np.pi/2),  # Joint 2
    (-np.pi/2, np.pi/2),  # Joint 3
    (-np.pi, np.pi/3),  # Joint 4
    (-np.pi/2, np.pi),  # Joint 5
    (-np.pi, np.pi)   # Joint 6
]
trr=tp(urdf_path=urdf_file_path,serial_manipulator=ur5,dynamics=ur5_d,joint_limits=Joint_limits,torque_limits=None)
traj = trr.joint_trajectory(thetastart=[0]*6,thetaend=thetalist,Tf=5,N=1000,method=5)

#tp.plot_trajectory(trajectory_data=traj,Tf=10)

2025-04-14 16:00:23 - ManipulaPy.path_planning - INFO - Generating joint trajectory.
2025-04-14 16:00:23 - numba.cuda.cudadrv.driver - INFO - init
/home/abo/.local/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
2025-04-14 16:00:25 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24 bytes
2025-04-14 16:00:25 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24 bytes
2025-04-14 16:00:25 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes
2025-04-14 16:00:25 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes
2025-04-14 16:00:25 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes


In [8]:
positions = traj['positions']
velocities = traj['velocities']
accelerations = traj['accelerations']
traj_tau = trr.inverse_dynamics_trajectory(positions,velocities,accelerations)


/home/abo/.local/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24000 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 24 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - add pending dealloc: cuMemFree_v2 864 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - dealloc: cuMemFree_v2 24 bytes
2025-04-14 16:00:26 - numba.cuda.cudadrv.driver - INFO - dealloc: cuMemFree_v2 24 byte